In [31]:
import pandas as pd
import open3d as o3d
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt

# # Get a list of all CSV file paths in the directory
# csv_files = glob.glob("drone_flight_dataset_dedrone/*.csv")

# # Read each CSV file and concatenate the dataframes
# dfs = []
# for file in csv_files:
#     df = pd.read_csv(file)
#     dfs.append(df)

# # Concatenate all dataframes into a single dataframe
# df = pd.concat(dfs, ignore_index=True)

# read in drone_flight_dataset_dedrone/2023-05-01_00-00-00_clean.csv
df = pd.read_csv('drone_flight_dataset_dedrone/2023-05-01_00-00-00_clean.csv')

df['SensorTime'] = pd.to_datetime(df['SensorTime'], format='mixed')

# Get unique drone IDs
drone_ids = df['DroneId'].unique()

print(len(drone_ids))

# Loop through each drone ID and assign a flight number to each unique flight
for drone_id in drone_ids:
    # Filter the dataframe for the current drone ID
    filtered_df = df[df['DroneId'] == drone_id]

    # sort filtered_df by SensorTime
    filtered_df = filtered_df.sort_values('SensorTime')

    # Convert the SensorTime column to numpy array
    timestamps = filtered_df['SensorTime'].values

    # Calculate the time differences between consecutive timestamps
    time_diff = np.diff(timestamps)

    # Find the indices where the time difference is greater than 30 minutes
    split_indices = np.where(time_diff > np.timedelta64(20, 'm'))[0] + 1

    # Assign flight IDs based on the split indices
    flight_ids = np.zeros(len(timestamps), dtype=int)
    current_flight_id = 1
    for idx in split_indices:
        flight_ids[idx:] = current_flight_id
        current_flight_id += 1

    # Add the flight_ids column to the DataFrame
    filtered_df['FlightNum'] = flight_ids
    df.loc[filtered_df.index, 'FlightNum'] = filtered_df['FlightNum']

# Create a new column called UniqueFlightId by concatenating DroneId and FlightNum
df['UniqueFlightId'] = df['DroneId'] + '_' + df['FlightNum'].astype(int).astype(str)


1456


We now have unique flight paths. Filter out those with fewer than 100 points

In [ ]:
# Loop through each flight ID
flight_ids = df['UniqueFlightId'].unique()

for flight_id in flight_ids:
    # Filter the dataframe for the current flight ID
    filtered_df_flight = df[df['UniqueFlightId'] == flight_id]

    if len(filtered_df_flight) < 100:
        df = df.drop(filtered_df_flight.index)
        continue

    # Describe the SensorTime column
    print(filtered_df_flight['SensorTime'].describe())

    # Plot the time series data with color based on FlightId
    plt.scatter(filtered_df_flight['DetectionLatitude'], filtered_df_flight['DetectionLongitude'], c=filtered_df_flight['Altitude'], cmap='viridis', s=10)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(f'Altitude vs. Time (Flight ID: {flight_id})')
    plt.show()

print(len(df['UniqueFlightId'].unique()))

# Generate synthetic flight data

In [3]:
import json
import pandas as pd
import numpy as np
import geopandas as gpd
# Load GeoJSON data
truedata_df = gpd.read_file('drug.geojson')

# Step 1: Generate a sequence of numbers for latitude and longitude
latitudes = truedata_df['geometry'].x  # Replace 30, 40 with the range of latitudes you want
longitudes = truedata_df['geometry'].y  # Replace -100, -90 with the range of longitudes you want

# Step 2: Generate random altitudes
altitudes = np.random.uniform(100, 500, 51)  # Replace 100, 500 with the range of altitudes you wants

# Step 3: Generate a sequence of timestamps
timestamps = pd.date_range('2024-05-04', periods=51, freq='S')  # Replace '2022-01-01' with the start date you want

# Step 4: Generate random drone types and drone IDs
drone_types = np.random.choice(['Mavic Mini'], 51)  # Replace 'Type1', 'Type2', 'Type3' with the drone types you have
drone_ids = np.random.choice(['eiue83jecf1d243944915ed78b4cehen8'], 51)  # Replace 1, 11 with the range of drone IDs you have

# Step 5: Generate flight numbers and unique flight IDs
unique_flight_ids = np.ones(51)  # 1000 ones

# Step 6: Combine all these arrays into a DataFrame
df = pd.DataFrame({
    'DetectionLatitude': latitudes,
    'DetectionLongitude': longitudes,
    'Altitude': altitudes,
    'SensorTime': timestamps,
    'DroneType': drone_types,
    'DroneId': drone_ids,
    'UniqueFlightId': unique_flight_ids
})
good_drone_df = df.copy()


# create the noisy DataFrames
noisy_dfs = []  # List to hold the noisy DataFrames
for i in range(1, 6):
    # Create a noisy version of good_drone_df for a different drone_id and flight_id
    noisy_drone_df = good_drone_df.copy()
    noisy_drone_df['DetectionLatitude'] = noisy_drone_df['DetectionLatitude'] + np.random.normal(0, 0.001, len(noisy_drone_df))
    noisy_drone_df['DetectionLongitude'] = noisy_drone_df['DetectionLongitude'] + np.random.normal(0, 0.001, len(noisy_drone_df))
    noisy_drone_df['Altitude'] = noisy_drone_df['Altitude'] + np.random.normal(0, 0.1, len(noisy_drone_df))
    noisy_drone_df['SensorTime'] = noisy_drone_df['SensorTime'] + pd.to_timedelta(np.random.normal(0, 1, len(noisy_drone_df)), unit='s')
    noisy_drone_df['DroneId'] = 'eiue83jecf1d243944915ed78b4cehen' + str(i + 8)  # Change the drone ID
    noisy_drone_df['UniqueFlightId'] = i + 1  # Change the flight ID

    # Append the noisy DataFrame to the list
    noisy_dfs.append(noisy_drone_df)

# Combine good_drone_df and all the noisy DataFrames
df = pd.concat([good_drone_df] + noisy_dfs)


/var/folders/my/znydn6fx7kscknpk2sdn5h3h0000gp/T/ipykernel_26781/2743420716.py:16: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  timestamps = pd.date_range('2024-05-04', periods=51, freq='S')  # Replace '2022-01-01' with the start date you want


In [7]:
import geopandas as gpd
# save the df to a new csv file
# df.to_csv('synthetic_data.csv', index=False)

print(df.head())

# Create a GeoDataFrame from the latitude and longitude columns
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['DetectionLongitude'], df['DetectionLatitude']))

# Convert the GeoDataFrame to GeoJSON format
geojson = gdf.to_file('synthetic_data.geojson', driver='GeoJSON')

# Print the GeoJSON
print(geojson)


   DetectionLatitude  DetectionLongitude    Altitude          SensorTime  \
0        -122.419400           37.774900  307.105586 2024-05-04 00:00:00   
1        -122.417680           37.776602  193.381144 2024-05-04 00:00:01   
2        -122.415960           37.778305  112.356850 2024-05-04 00:00:02   
3        -122.414240           37.780007  367.805778 2024-05-04 00:00:03   
4        -122.412519           37.781710  426.160286 2024-05-04 00:00:04   

    DroneType                            DroneId  UniqueFlightId  
0  Mavic Mini  eiue83jecf1d243944915ed78b4cehen8             1.0  
1  Mavic Mini  eiue83jecf1d243944915ed78b4cehen8             1.0  
2  Mavic Mini  eiue83jecf1d243944915ed78b4cehen8             1.0  
3  Mavic Mini  eiue83jecf1d243944915ed78b4cehen8             1.0  
4  Mavic Mini  eiue83jecf1d243944915ed78b4cehen8             1.0  
None


In [52]:
import matplotlib.cm as cm

# Create a point cloud object
point_cloud = o3d.geometry.PointCloud()

# Set the point cloud coordinates
point_cloud.points = o3d.utility.Vector3dVector(df[['DetectionLatitude', 'DetectionLongitude', 'Altitude']].values)

# Step 1: Normalize the UniqueFlightId column to be between 0 and 1
df['NormalizedFlightId'] = (df['UniqueFlightId'] - df['UniqueFlightId'].min()) / (df['UniqueFlightId'].max() - df['UniqueFlightId'].min())

# Step 2: Use the normalized UniqueFlightId to create a colormap
colors = cm.get_cmap('rainbow')(df['NormalizedFlightId'])

# Step 3: Convert the colormap to a Vector3dVector and assign it to the colors attribute of your point cloud
point_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])

# Visualize the point cloud
o3d.visualization.draw_geometries([point_cloud])